In [1]:
from keras.datasets import fashion_mnist

(x_train, y_train) , (x_test, y_test) = fashion_mnist.load_data() 

Using TensorFlow backend.


4423680/4422102 [==============================] - 4s 1us/step


In [2]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False)
y_train_encodded = encoder.fit_transform(y_train.reshape(len(y_train), 1))
y_test_encodded = encoder.transform(y_test.reshape(len(y_test), 1))

print('y_train_encodded.shape={}'.format(y_train_encodded.shape))
print('y_test_encodded.shape={}'.format(y_test_encodded.shape))

y_train_encodded.shape=(60000, 10)
y_test_encodded.shape=(10000, 10)


In [33]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, GlobalAveragePooling2D, Reshape
from keras.layers import BatchNormalization, Dropout, MaxPooling2D
from keras.optimizers import Adam, SGD

class CNN(object):
    def __init__(self):
        self.model = self.build_model()
        
    
    def build_model(self, activation='relu', kernel_size=(3, 3), strides=1, pool_size=(2, 2)):
        model = Sequential()
        
        # reshape
        model.add(Reshape((28, 28, 1), input_shape=(28, 28)))
        
        # Conv2D(num_filters, ...), Output shape=(None, 14, 14, 32)
        model.add(Conv2D(32, kernel_size=kernel_size, 
                         strides=strides, padding='same', activation=activation,
                         ))
        model.add(Conv2D(32, kernel_size=kernel_size, 
                         strides=strides, padding='same', activation=activation,
                         ))
        
        model.add(MaxPooling2D(pool_size=pool_size))
        model.add(BatchNormalization())
        model.add(Dropout(0.25))
        
        # Output shape = (None, 7, 7, 64)
        model.add(Conv2D(64, kernel_size=kernel_size, 
                         strides=strides, padding='same', activation=activation))
        model.add(Conv2D(64, kernel_size=kernel_size, 
                         strides=strides, padding='same', activation=activation))
        model.add(MaxPooling2D(pool_size=pool_size))
        model.add(BatchNormalization())
        model.add(Dropout(0.25))
        
        
        # Output shape = (None, 3, 3, 128)
        model.add(Conv2D(128, kernel_size=kernel_size,
                        strides=strides, padding='valid', activation=activation))
        model.add(Conv2D(128, kernel_size=kernel_size,
                        strides=strides, padding='valid', activation=activation))
        model.add(MaxPooling2D(pool_size=pool_size))
        model.add(BatchNormalization())
        model.add(Dropout(0.25))
        
        # Output shape= (None, 128)
        model.add(GlobalAveragePooling2D())
        model.add(BatchNormalization())
        model.add(Dropout(0.25))
        
        # Output shape = (None, 128)
        model.add(Dense(128, activation=activation))
        model.add(BatchNormalization())
        model.add(Dropout(0.25))
        
        # Output shape = (None, 64)
        model.add(Dense(64, activation=activation))
        model.add(BatchNormalization())
        model.add(Dropout(0.25))
        
        model.add(Dense(10, activation='softmax'))
        
        ### define optimizer
        sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
#         model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
        
        return model
    
    
    def train(self, X_train, y_train, X_val, y_val, epochs, batch_size=32):
        self.model.fit(X_train, y_train, 
                      epochs=epochs,
                       batch_size=batch_size,
                       verbose=1,
                      validation_data=(X_val, y_val))
        

    def score(self, X, y):
        return self.model.evalution(X, y, verbose=0)

In [34]:
cnn = CNN()
cnn.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_10 (Reshape)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
batch_normalization_50 (Batc (None, 14, 14, 32)        128       
_________________________________________________________________
dropout_35 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 14, 14, 64)        18496     
__________

In [39]:
epochs = 4  # previously have done 6 epochs. Total epochs = 10

cnn.train(x_train, y_train_encodded, x_test, y_test_encodded, epochs)

Train on 60000 samples, validate on 10000 samples
Epoch 1/4
60000/60000 [==============================] - 44s 731us/step - loss: 0.2972 - acc: 0.8959 - val_loss: 0.2629 - val_acc: 0.9048
Epoch 2/4
60000/60000 [==============================] - 43s 716us/step - loss: 0.2822 - acc: 0.9008 - val_loss: 0.2482 - val_acc: 0.9097
Epoch 3/4
60000/60000 [==============================] - 42s 707us/step - loss: 0.2755 - acc: 0.9037 - val_loss: 0.2380 - val_acc: 0.9152
Epoch 4/4
60000/60000 [==============================] - 43s 710us/step - loss: 0.2648 - acc: 0.9083 - val_loss: 0.2333 - val_acc: 0.9159
